# Distributed Scheduling Deep-dive

## Ray Cluster overview

A Ray cluster consits of:
- One or more **woker nodes**, where each worker node consists of the following processes:
    - **worker processes** responsible for task submission and execution. Each worker process stores:
        - An **ownership table**: System metadata for the objects to which the worker has a reference, e.g., to store ref counts and object locations
        - An **in-process store**: used to store small objects (<100KB)
    - A **raylet**: Manages shared resources on each node. The raylet has two main components:
        - A **scheduler**: Responsible for resource management, task placement, and fulfilling task arguments that are stored in the distributed object store. The individual schedulers in a cluster comprise **the Ray distributed scheduler**.
        - A **shared-memory object store** (also known as the **Plasma Object Store**). Responsible for storing, transferring, and spilling **large objects**. The individual object stores in a cluster comprise the **Ray distributed object store**.
- One of the worker nodes is designated as a **head node** which is a special node that also hosts
  - A **global control service**: keeps track of the **cluster state** that is not supposed to change often
  - **Cluster level services**: are services that are shared across the cluster suc as autoscaling, job submission, etc. 
  - A **driver**: a special worker process that executes the top-level application. It can submit tasks but does not execute them.

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/ray_cluster.png" width="800">


Note that 
- The **plasma object store** by default is in-memory and takes up **30% of the memory of the node**
- If the **plasma object store** is full, objects are **spilled to disk**

<!-- 
Reference: 
- See [V2 architecture document -> Architecture Overview -> Design -> Components](https://docs.google.com/document/d/1tBw9A4j62ruI5omIJbMxly-la5w4q_TjyJgJL_jN2fI/preview#heading=h.cclei73t0j5p)
 -->

## Scheduling a Task

Below is a high-level diagram showing the primary stages in scheduling a task leading to its execution. 

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_high_level.svg" width="1400px">

Note that the diagram presents a simplified ordering of the steps invovled. Also note that while it shows certain steps being launched from a worker or a raylet, those steps are run in separate processes as we will detail later.


### Stage 1: Serializing Function Code

When a Ray task is first called, its definition is pickled and then stored in the GCS.

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_serialization.svg" width="600px">


<!-- References:
- See code:
    1. [Python .remote calls ._remote](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/remote_function.py#L140)
    2. [Python ._remote pickles the function](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/remote_function.py#L299)
    3. [Python ._remote call exports the function via the function manager.export](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_private/function_manager.py#L273)
    4. [Which calls the cython GcsClient.internal_kv_put](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_raylet.pyx#L2579)
    5. [Which calls the gcs_client.cc PythonGcsClient::InternalKVPut](https://github.com/ray-project/ray/blob/55ab6dfd6b415f8795dd1dfed7b3fde2558efc46/src/ray/gcs/gcs_client/gcs_client.cc#L312) that sets the key, value in the proper namespace -->

### Stage 2: Claiming ownership

Ownership is claimed by following these steps:
- The core worker process will request from its task manager to add the pending task
- The task manager will create an object reference for the returned result
- The core worker will request from its reference counter to update its ownership table with the created object reference

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_ownership_claim.svg" width="600px">

<!-- References:
- See code:
    1. [Python .remote calls ._remote](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/remote_function.py#L140)
    2. [python ._remote call calls submit_task](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/remote_function.py#L420)
    3. [submit_task calls the cython submit_task function defined here](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_raylet.pyx#L3574)
    4. [cython submit_task Delegates to C++ CoreWorker::SubmitTask]((https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_raylet.pyx#L3643)
    5. [CoreWorker::SubmitTask in turn calls TaskManager::AddPendingTask](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/core_worker.cc#L1944)
    6. [TaskManager::AddPendingTask assigns a return id for referencing the task result](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/task_manager.cc#L195]
    7. [TaskManager::AddPendingTask calls ReferenceCounter::AddOwnedObject to claim ownership of task result](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/task_manager.cc#L207)
  -->

### Stage 3: Resolving Task Dependencies

Given a particular task `Task` that depends on:
- object `A` as input
- object `B` as input

The core worker's submitter process will perform these steps

1. Find each dependency's owner:
    - Inspect the object reference to object `A` to find its owner address (IP + port)
    - Inspect the object reference to object `B` to find its owner address (IP + port)
2. Fetch metadata from the owner's ownership table about each object
    - Request metadata about object `A` (e.g. the size and location of the of the object)
    - Request metadata about object `B` (e.g. the size and location of the of the object)
3. Proceed with scheduling now that all dependencies are resolved

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_resolving_deps.svg" width="600px">

<!-- References:
- See code:
    1. [Python .remote calls ._remote](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/remote_function.py#L140)
    2. [python ._remote call calls submit_task](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/remote_function.py#L420)
    3. [submit_task calls the cython submit_task function defined here](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_raylet.pyx#L3574)
    4. [cython submit_task Delegates to C++ CoreWorker::SubmitTask]((https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_raylet.pyx#L3643)
    5. [CoreWorker::SubmitTask calls direct_task_submitter.SubmitTask](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/core_worker.cc#L1949)
    5. [direct_task_submitter.SubmitTask calls ResolveDependencies to resolve dependencies](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/transport/direct_task_transport.cc#L28)
    6. [ResolveDependencies calls InlineDependencies](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/transport/dependency_resolver.cc#L117)
    7. [InlineDependencies fetches task metadata like the size](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/transport/dependency_resolver.cc#L44C10-L44C10)
  -->

### Stage 4: Enforcing data locality

The core worker's submitter process will choose the raylet on the node that has the most number of object argument bytes already local to schedule the task.

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_data_locality.svg" width="800px">

Note: this stage is skipped in case the task's specified scheduling policy is stringent (e.g. a node-affinity policy)

<!-- References:
- See code:
    1. [Python .remote calls ._remote](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/remote_function.py#L140)
    2. [python ._remote call calls submit_task](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/remote_function.py#L420)
    3. [submit_task calls the cython submit_task function](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_raylet.pyx#L3643)
    4. [cython submit_task Delegates to SubmitTask from c++ Core Worker with calls direct_task_submitter.SubmitTask](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/core_worker.cc#L1949)
    5. [direct_task_submitter.SubmitTask calls ResolveDependencies to resolve dependencies](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/transport/direct_task_transport.cc#L28)
    6. [direct_task_submitter.SubmitTask as a callback will now call RequestNewWorkerIfNeeded](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/transport/direct_task_transport.cc#L135)
    7. [RequestNewWorkerIfNeeded will in turn call GetBestNodeForTask](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/transport/direct_task_transport.cc#L394)
    8. [GetBestNodeForTask will pick a node for locality in case the scheduling strategy is not stringent (i.e. node affinity or spread)](https://github.com/ray-project/ray/blob/master/src/ray/core_worker/lease_policy.cc#L39)
    9. [GetBestNodeIdForTask will find the node with the most object bytes](https://github.com/ray-project/ray/blob/master/src/ray/core_worker/lease_policy.cc#L47C1-L48C1) -->

### Stage 5: Applying Scheduling Policy

The selected "Scheduler Raylet" will undergo these steps to locate the best node to execute the task

- It receives updates from the GCS every 100ms
    - The updates contain metadata about the resources on each node
- It determines which nodes/raylets are feasible for scheduling given the task resource requirements
- It will then apply the task's scheduling policy
- The policy will determine the node/raylet that will be executing the task (i.e. our "executor raylet")


<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_applying_policy.svg" width="600px">

<!-- References:
- See code:
    1. Add task to task queue:
        1. [Python .remote calls ._remote](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/remote_function.py#L140)
        2. [python ._remote call calls submit_task](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/remote_function.py#L420)
        3. [submit_task calls the cython submit_task function](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_raylet.pyx#L3643)
        4. [cython submit_task Delegates to SubmitTask from c++ Core Worker with calls direct_task_submitter.SubmitTask](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/core_worker.cc#L1949)
        5. [direct_task_submitter.SubmitTask calls ResolveDependencies to resolve dependencies](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/transport/direct_task_transport.cc#L28)
        6. [direct_task_submitter.SubmitTask will now schedule the task onto the task queue](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/transport/direct_task_transport.cc#L113)
    2. Raylet's scheduler picks up task from task queue and applies scheduling policy
        3. [A raylet when instantiated is composed of a node manager](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/raylet.h#L96)
        4. A node manager is composed of:
            - A cluster task manager which is responsible for queuing, spilling back, and dispatching tasks.
            - A cluster resource scheduler is responsible for maintaining a view of the cluster state w.r.t resource usage.
            - [These two classes make up the distributed scheduler](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/node_manager.h#L819)
        5. [direct_task_submitter.SubmitTask as a callback will now call RequestNewWorkerIfNeeded](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/transport/direct_task_transport.cc#L135)
        6. [RequestNewWorkerIfNeeded will in turn call RequestWorkerLease](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/transport/direct_task_transport.cc#L405)
        6. [When a worker lease request comes in a raylet's NodeManager::HandleRequestWorkerLease gets call](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/node_manager.cc#L1783)
        7. [NodeManager::HandleRequestWorkerLease will delegate a call to ClusterTaskManager::QueueAndScheduleTask()](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/node_manager.cc#L1830)
        8. [ClusterTaskManager::QueueAndScheduleTask it will delegate a call to ClusterTaskManager::ScheduleAndDispatchTasks()](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/scheduling/cluster_task_manager.cc#L64)
        9. [ScheduleAndDispatchTasks will call ClusterResourceManager::GetBestSchedulableNode() taking into account the schedulding strategy set on the task specification](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/scheduling/cluster_task_manager.cc#L148C14-L155C1) -->

### Stage 6: Leasing a worker

The executor raylet will perform these steps for each task in its queue
- It will try to ensure a fair distribution of tasks amongst different scheduling policies
- It will wait for the tasks arguments to be available
- It will check if the node can allocate resources for the task
- It will now secure a worker and respond with the worker address to the owner process

If it fails to any of the checks, it will try to:
- wait and retry the steps
- re-apply the scheduling policy to check if it can spill the task to another node that is now better suited

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_leasing_worker.svg" width="1000px">

Notes: 
- After this stage, the owner process will now own the lease to the worker
- The lease remains active as long as the owner process and leased worker are alive, and the raylet ensures that no other client may use the worker while the lease is active.
- The owner may schedule any number of tasks onto the leased worker, as long as the tasks are compatible with the granted resource request. Hence, leases can be thought of as an optimization to avoid communication with the scheduler for similar scheduling requests.


<!-- References:
- See code:
    1. [When a worker lease request comes in a raylet's NodeManager::HandleRequestWorkerLease gets call](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/node_manager.cc#L1783)
    2. [NodeManager::HandleRequestWorkerLease will delegate a call to ClusterTaskManager::QueueAndScheduleTask()](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/node_manager.cc#L1830)
    3. [ClusterTaskManager::QueueAndScheduleTask it will delegate a call to ClusterTaskManager::ScheduleAndDispatchTasks()](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/scheduling/cluster_task_manager.cc#L64)
    4. [ClusterTaskManager.ScheduleAndDispatchTasks() will call LocalTaskManager::ScheduleAndDispatchTasks()](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/scheduling/cluster_task_manager.cc#L227)
    5. [LocalTaskManager::ScheduleAndDispatchTasks() will call LocalTaskManager::DispatchScheduledTasksToWorkers()](https://github.com/ray-project/ray/blob/master/src/ray/raylet/local_task_manager.cc#L96)
    6. [LocalTaskManager::DispatchScheduledTasksToWorkers() will secure a worker for a task in the call to WorkPool.PopWorker() only after securing that owner is active, arguments are available, resources are allocated)](https://github.com/ray-project/ray/blob/master/src/ray/raylet/local_task_manager.cc#L267)
-->

### Stage 7: Executing the task

Once a worker receives a task it will perform these steps:

- Consume the task from the queue
- Prepare the function for execution
- Submit the RayFunction to a task execution callback that will
    - Deserialize the function arguments
    - Construct and execute the python function within cython (dealing with async functions)
    - Handle results properly:
        - Perform error handling extracting error information if there is a failure
        - Deal with generator results
    - Prepare the return object
        - Seal the return object
        - Update the reference counts
        - Run garbage collection:
            - if "borrowed" dependency objects are no longer needed, they will be deleted from the object store
        - If the return object(s) are small
            - Return the values inline directly to the owner, which copies them to its in-process object store.
        - If the return object(s) are large
            - Store the objects in the node's shared memory store and reply to  owner indicating the objects are now in distributed memory.
- Return the final status of the task execution



<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_execution.svg" width="1200px">


<!-- References: 
- See code:
    1. [When instantiating a CoreWorker, we add task receivers which will callback CoreWorker::ExecuteTask](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/core_worker.cc#L147)
    2. [CoreWorker::ExecuteTask() will prepare a RayFunction and submit it to its execution callback](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/core_worker/core_worker.cc#L2721C21-L2721C44)
    3. [The task execution callback in the case of python will execute the function from cython given the set task_execution_handler](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_raylet.pyx#L3075C43-L3075C65)
    4. [The task execution handler will execute the task with a cancellation handler](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_raylet.pyx#L2064C17-L2064C55)
    5. [The handler will call execute_task handling a KeyboardInterrupt error](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_raylet.pyx#L1960C7-L1960C7)
    6. [execute_task will invoke the function_executor](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_raylet.pyx#L1675)
    7. [execute_task will store the outputs in the object store](https://github.com/ray-project/ray/blob/releases/2.8.1/python/ray/_raylet.pyx#L1810C12-L1810C12) -->

### Stage 8: Fetching the task result

Here are the steps in fetching a task's result, i.e. when ray.get gets called

1. find the owner using the object reference
2. the owner will look up its ownership table to find where in distributed memory the object is located
3. the owner will fetch the object and return it
4. the owner will update the reference count to the object
5. the owner will trigger grabage collection if the reference count is now 0


<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_fetching.svg" width="800px">

# Overview of Scheduling Strategies

Ray provides different scheduling strategies that you can set on your task.

We will go over:
- How a raylet assess feasibility and availability of nodes
- How every scheduling strategy works and when you should use it

## How does a raylet classify nodes as feasible/infeasible and available/unavailable?

Given a resource requirement, a raylet will determine if a node is
- feasible vs infeasible node 
- for feasible nodes, the raylet determine if the node is:
    - available or not available

The raylet will make use of the resource updates that it receives by broadcast from GCS every 100ms to make these decisions.

Let's understand this by looking at this example task `Task`, that has a resource requirement of 3 CPUs:
- all nodes with >= 3 CPUs are classified as **feasible**, otherwise infeasible
    - for those nodes that have >= 3 CPUs **idle**, they are classified as **available**, otherwise not available

In [6]:
import pandas as pd

# Read DataFrame from CSV
pd.read_csv("https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/raylet_node_classification.csv", header=[0, 1])

Task              Node  \
  Resource Requirements Resource Capacity   
0            num_cpus=3            4 CPUs   
1            num_cpus=3            4 CPUs   
2            num_cpus=3             2CPUs   

                                                            Raylet  
                                Status              Classification  
0                            Idle node      Feasible and Available  
1  2 CPUs tied up running another task  Feasible but Not Available  
2                            Idle node                  Infeasible

## Default Scheduling Strategy

### How does it work?
This is the default policy used by ray. It is a hybrid policy that combines the following two heuristics:
- Bin packing heuristic
- Load balancing heuristic

### Use-cases
- Works well as a default: it usually results in data locality being honored.

<!-- ### References:
- See code here:
    - [Default Hybrid Scheduling Policy is defined here](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/scheduling/policy/hybrid_scheduling_policy.cc) -->

The diagram below shows the policy in action in a bin-packing heuristic/mode

Note the **Local Node** shown in the diagram is the node that is local to the raylet that received the worker lease request - which in almost all cases is the raylet that satisfies data locality requirements.

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_policy_hybrid_policy_binpacking.svg" width="900px">

The diagram below shows the policy in action in a load balancing heuristic. 

This occurs when our preferred local node is heavily being utilized. The strategy will now spread new tasks amongst other feasible and available nodes.

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_policy_hybrid_policy_balancing.svg" width="900px">

## Node Affinity Strategy

### How does it work?
It assigns a task to a given node in either a strict or soft manner.

### Use-cases
- When you want to ensure that your task runs on a specific node: e.g. you want to make sure a given accelerator is used for a compute-intensive task.


<!-- ### References:
- See code here
    - [Node Affinity Policy is defined here](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/scheduling/policy/node_affinity_scheduling_policy.cc)
  -->

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_policy_node_affinity.svg" width="1000px">

### Sample code

In [1]:
import ray
from ray.util.scheduling_strategies import NodeAffinitySchedulingStrategy


@ray.remote(
    scheduling_strategy=NodeAffinitySchedulingStrategy(
        node_id=ray.get_runtime_context().get_node_id(),
        soft=False,
    )
)
def node_affinity_schedule():
    return 2


ray.get(node_affinity_schedule.remote())

2023-12-18 15:03:45,916	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


2

## SPREAD Scheduling Strategy

### How does it work?
It behaves like a best-effort round-robin. It spreads across all the available nodes first and then the feasible nodes.

### Use-cases
- When you want to load-balance your tasks across nodes. e.g. you are building a web service and want to avoid overloading certain nodes.


<!-- ### References:
- See code here
    - [Spread Scheduling Policy is defined here](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/scheduling/policy/spread_scheduling_policy.cc)
  -->

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_policy_spread.svg" width="700px">

### Sample Code

In [2]:
import ray


@ray.remote(scheduling_strategy="SPREAD")
def spread_default_func():
    return 2


ray.get(spread_default_func.remote())

2

## Placement Group Scheduling Strategy

In cases when we want to treat a set of resources as a single unit, we can use placement groups.

### How does it work?

- A **placement group** is formed from a set of **resource bundles**
  - A **resource bundle** is a list of resource requirements that fit in a single node
- A **placement group** can specify a **placement strategy** that determines how the **resource bundles** are placed
  - The **placement strategy** can be one of the following:
    - **PACK**: pack the **resource bundles** into as few nodes as possible
    - **SPREAD**: spread the **resource bundles** across as many nodes as possible
    - **STRICT_PACK**: pack the **resource bundles** into as few nodes as possible and fail if not possible
    - **STRICT_SPREAD**: spread the **resource bundles** across as many nodes as possible and fail if not possible
- **Placement Groups** are **atomic** 
  -  i.e. either all the **resource bundles** are placed or none are placed
  -  GCS uses a two-phase commit protocol to ensure atomicity

### Use-cases
- Use SPREAD when you want to load-balance your tasks across nodes. e.g. you are building a web service and want to avoid overloading certain nodes.
- Use PACK when you want to maximize resource utilization. e.g. you are running training and want to cut costs by packing all your resource bundles on a small subset of nodes.

<!-- ### References
- [See code here for Bundle Scheduling Policy](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/raylet/scheduling/policy/bundle_scheduling_policy.cc) -->

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/scheduling_policy_placement_group.svg" width="600px">

### Example Code

In [4]:
import ray
from ray.util.scheduling_strategies import PlacementGroupSchedulingStrategy
# Import placement group related functions
from ray.util.placement_group import (
    placement_group,
    placement_group_table,
    remove_placement_group,
)

# Reserve a placement group of 1 bundle that reserves 0.1 CPU
pg = placement_group([{"CPU": 0.1}], strategy="PACK", name="my_pg")

# Wait until placement group is created.
ray.get(pg.ready(), timeout=10)

# look at placement group states using the table
print(placement_group_table(pg))


@ray.remote(
    scheduling_strategy=PlacementGroupSchedulingStrategy(
        placement_group=pg,
    ),
    # task requirement needs to be less than placement group capacity
    num_cpus=0.1,
)
def placement_group_schedule():
    return 2


out = ray.get(placement_group_schedule.remote())
print(out)

# Remove placement group.
remove_placement_group(pg)

{'placement_group_id': '1f68b1aaa61cf3a4130f9bd5ec2c01000000', 'name': 'my_pg', 'bundles': {0: {'CPU': 0.1}}, 'bundles_to_node_id': {0: 'ed50ecd2dcb4443e107b5b0b9da78065e86e8428924fd1ae82c0a780'}, 'strategy': 'PACK', 'state': 'CREATED', 'stats': {'end_to_end_creation_latency_ms': 2.57, 'scheduling_latency_ms': 2.349, 'scheduling_attempt': 1, 'highest_retry_delay_ms': 0.0, 'scheduling_state': 'FINISHED'}}
2
